In [ ]:
import modules.functions as fn
import os
import pandas as pd
import glob
import json
import webbrowser
import tempfile
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

##### === CONFIGURACIÓN DE ESTILOS ===


In [ ]:
def get_table_styles():
    return """
    <style>
        /* Reset de estilos para Outlook */
        table {
            border-collapse: collapse !important;
            width: 100% !important;
            margin-bottom: 20px !important;
            mso-table-lspace: 0pt !important;
            mso-table-rspace: 0pt !important;
            font-size: 11px !important;
        }
        th, td {
            border: 1px solid #ddd !important;
            padding: 2px 4px !important;  /* Reducido el padding vertical a 2px */
            text-align: left !important;
            font-family: Arial, sans-serif !important;
            font-size: 11px !important;
            mso-line-height-rule: exactly !important;
            line-height: 1.2 !important;  /* Altura de línea reducida */
            height: 16px !important;      /* Altura fija para las celdas */
        }
        th {
            background-color: #cccccc !important;
            color: white !important;
            font-weight: bold !important;
            height: 18px !important;      /* Altura ligeramente mayor para encabezados */
        }
        tr {
            height: 16px !important;      /* Altura fija para las filas */
        }
        tr:nth-child(even) {
            background-color: #f2f2f2 !important;
        }
        /* Contenedor de tabla para layout lado a lado */
        .table-cell {
            width: 48% !important;
            padding: 5px !important;
            vertical-align: top !important;
        }
        /* Clase específica para las tablas de datos */
        .data-table {
            margin: 0 !important;
            padding: 0 !important;
        }
        .data-table td {
            height: 16px !important;
            max-height: 16px !important;
            overflow: hidden !important;
            white-space: nowrap !important;
        }
    </style>
    """

##### === FUNCIONES DE ESTILIZACIÓN ===

In [ ]:
# Definir la función para colorear las celdas en la columna "Score"
def color_score(val):
    if val == 0:
        return 'background-color: #b3e6b3; color: #2d6a2d'  # Verde pastel
    elif 0 > val >= -20:
        return 'background-color: #fff2b3; color: #7a7a00'  # Amarillo pastel
    elif val < -20:
        return 'background-color: #f7b3b3; color: #a10000'  # Rojo pastel
    return ''

In [ ]:
# Definir la función para colorear la celda en la columna "Num" de la fila "FAILED" en la segunda tabla
def color_failed(row):
    if row['STATUS'] == 'Failed' and row['Num'] != 0:
        return ['background-color: #f7b3b3; color: #a10000'] * len(row)
    return [''] * len(row)

In [ ]:
# Definir la función para colorear las celdas en la columna "status" de la tercera tabla
def color_status(val):
    if val == "GOOD":
        return 'background-color: #b3e6b3; color: #2d6a2d'  # Verde pastel
    elif val == "FAIR":
        return 'background-color: #fff2b3; color: #7a7a00'  # Amarillo pastel
    elif val == "POOR":
        return 'background-color: #f7b3b3; color: #a10000'  # Rojo pastel
    return ''

In [ ]:
# Definir la función para colorear las celdas en la columna "READINESS"
def color_readiness(val): 
    if val == "ready": 
        return 'background-color: #b3e6b3; color: #2d6a2d'  # Verde pastel
    elif val == "migrating": 
        return 'background-color: #fff2b3; color: #7a7a00'  # Amarillo pastel
    elif val == "not_ready": 
        return 'background-color: #f7b3b3; color: #a10000'  # Rojo pastel
    return ''

In [ ]:
# Definir la función de estilo para colorear las celdas de la columna "Rate (%)"
def color_rate(val):
    if val == 100:
        return 'background-color: #b3e6b3; color: #2d6a2d'  # Verde pastel
    elif 90 <= val < 100:
        return 'background-color: #ffdab3; color: #7a4100'  # Naranja pastel
    elif 80 <= val < 90:
        return 'background-color: #fff2b3; color: #7a7a00'  # Amarillo pastel
    elif val < 80:
        return 'background-color: #f7b3b3; color: #a10000'  # Rojo pastel
    return ''

In [ ]:
# Definir la función de estilo para colorear las celdas de la columna "Rate (%)"
def color_health_status(val):
    if val == "GOOD":
        return 'background-color: #b3e6b3; color: #2d6a2d'  # Verde pastel
    elif val == "FAIR":
        return 'background-color: #fff2b3; color: #7a7a00'  # Amarillo pastel
    elif val == "POOR":
        return 'background-color: #f7b3b3; color: #a10000'  # Rojo pastel
    return ''

In [ ]:
# Mapear funciones de estilización por sistema
STYLE_FUNCTIONS = {
    "PPDM": {
        "Score": color_score,
        "STATUS": color_status,
        "failed": color_failed,
        "readiness": color_readiness,
        "Rate (%)": color_rate,
        "healthStatus": color_health_status
    },
    "DD": {"STATUS": color_status},  # Puedes agregar más columnas y estilos específicos
    "Avamar": {"Score": color_score}  # Ajustar según necesidades
}

##### === FUNCIONES PRINCIPALES ===

In [ ]:
def process_system_files(system, hostname, config):
    base_path = config['basePath']
    csv_path = os.path.join(base_path, config['csvPath'])
    file_patterns = config['systems'][system]['files']

    html_content = []
    
    for file_key, pattern in file_patterns.items():
        file_path = glob.glob(f"{csv_path}/{system}-{hostname}-{pattern}")
        if not file_path:
            continue
        
        df = pd.read_csv(file_path[0])
        
        # Aplicar estilos según las funciones definidas para el sistema
        style_functions = STYLE_FUNCTIONS.get(system, {})
        for column, style_func in style_functions.items():
            if column in df.columns:
                df.style.applymap(style_func, subset=[column])
        
        html_table = df.to_html(index=False, table_id=file_key, classes="data-table")
        html_content.append(html_table)
    
    return html_content

In [ ]:
def generate_html_report(config):
    html_body = f"""
    <html xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:w="urn:schemas-microsoft-com:office:word" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns="http://www.w3.org/TR/REC-html40">
    <head>
        {get_table_styles()}
        <!--[if gte mso 9]>
        <xml>
            <o:OfficeDocumentSettings>
                <o:AllowPNG/>
                <o:PixelsPerInch>96</o:PixelsPerInch>
            </o:OfficeDocumentSettings>
        </xml>
        <![endif]-->
    </head>
    <body style="margin: 0; padding: 0; font-family: Arial, sans-serif;">
        <div style="width: 100%; max-width: 1200px; margin: 0 auto;">
            <h2 style="font-family: Arial, sans-serif; color: #0044cc; margin-bottom: 10px;">DAILY CHECK PPDM</h2>
"""
    
    for system, system_config in config["systems"].items():
        for instance in system_config["instances"]:
            hostname = instance["hostname"]
            html_body += f"<h3>Hostname: {hostname} (System: {system})</h3>"
            
            # Procesar archivos y obtener tablas HTML
            tables = process_system_files(system, hostname, config)
            for table in tables:
                html_body += table
    
    html_body += "</body></html>"
    return html_body

##### === SCRIPT PRINCIPAL ===

In [ ]:
if __name__ == "__main__":
    config = fn.load_json_file("config_encrypted.json")
    html_content = generate_html_report(config)
    
    # Guardar HTML temporalmente
    with tempfile.NamedTemporaryFile("w", delete=False, suffix=".html") as f:
        f.write(html_content)
        webbrowser.open(f"file://{f.name}")
    
    # Enviar el correo (opcional)
    # send_email(config, html_content)